In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

In [2]:
newzy = pd.read_csv('../FinalProject/newzy.csv', sep='|', index_col='doc_id')

/Users/matthewthomas/anaconda3/lib/python3.7/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
#articles = pd.read_csv('../FinalProject/articles.csv', index_col=0)

In [3]:
#articles = articles['0'].to_list()

In [4]:
newzy.doc_source.unique()

array(['Google News', 'PowerLine', 'Reuters', 'Guardian', 'Daily Kos',
       'Drudge Report', 'Real Clear Politics', 'NPR', 'Fox',
       'New York Times', 'Breitbart', 'CNN', 'US News', 'UPI Latest',
       'Politico Magazine', nan], dtype=object)

In [5]:
filters =  ['Powerline', 'Daily Kos',\
            'Real Clear Politics', 'Politico Magazine']

In [6]:
newzy = newzy[newzy.doc_source.isin(filters)]

In [7]:
newzy

,doc_source,doc_title,doc_content,doc_date,doc_url
doc_id,,,,,
96,Daily Kos,Obama has done nothing to address income inequ...,Income and wealth inequality is a festering wo...,11/03/2013,http://feeds.dailykos.com/~r/dailykos/index/~3...
97,Daily Kos,"Anti-racism is not 'anti-white,' no matter wha...",The virulent meme spread across our nation wit...,11/03/2013,http://feeds.dailykos.com/~r/dailykos/index/~3...
98,Daily Kos,Abbreviated Pundit Round-up: Angry does no...,Chrystia Freeland sees a fight that's not not ...,11/03/2013,http://feeds.dailykos.com/~r/dailykos/index/~3...
99,Daily Kos,Sunday Talk: Haters gonna hate,"No doubt, the bestest thing about America is o...",11/03/2013,http://feeds.dailykos.com/~r/dailykos/index/~3...
100,Daily Kos,Gore on fire: Says humans treating atmosphere ...,About time Al Gore has been everywhere lately....,11/03/2013,http://feeds.dailykos.com/~r/dailykos/index/~3...
...,...,...,...,...,...
1026314,Real Clear Politics,"Bernie Sanders, the Sixty Trillion Dollar Man",The price of Bernie Sanders's agenda could be ...,02/27/2020,https://www.realclearpolitics.com/2020/02/26/b...
1026315,Real Clear Politics,Sanders' Plans Cheap Compared to Cost of Doing...,"Investing in healthcare, public higher educati...",02/27/2020,https://www.realclearpolitics.com/2020/02/26/s...
1026316,Real Clear Politics,Weinstein and the Complicated Legacy of Me Too,"When the guilty verdict was delivered, Hollywo...",02/27/2020,https://www.realclearpolitics.com/2020/02/26/w...


In [22]:
articles = []
def parse_text(links):
    for link in links:
        url = link
        try:
            r = requests.get(url) #request documentation for errors
        except requests.exceptions.RequestException:
            articles.append('bad link')
        else:
            html_doc = r.text
            soup = BeautifulSoup(html_doc) #check error here
            p_tags = soup.find_all('p')
            p_tags_text = [tag.get_text().strip() for tag in p_tags]
            #Some links don't work so we need to check for that
            if len(p_tags_text) == 0:
                articles.append('bad link')
            else:
                # Filter out sentences that contain newline characters '\n' or don't contain periods.
                sentence_list = [sentence for sentence in p_tags_text if not '\n' in sentence]
                sentence_list = [sentence for sentence in sentence_list if '.' in sentence]
                # Combine list items into string.
                article = ' '.join(sentence_list)
                articles.append(article)

In [23]:
''''def url_to_string(links):
    for url in links:
        res = requests.get(url)
        html = res.text
        soup = BeautifulSoup(html, 'html5lib')
        for script in soup(["script", "style", 'aside']):
            script.extract()
        article = " ".join(re.split(r'[\n\t]+', soup.get_text()))
        articles.append(article)

### Download Real Clear News Articles

In [6]:
realclear = newzy[newzy.doc_source == 'Real Clear Politics'].reset_index()

In [47]:
len(realclear)

40611

In [55]:
links = realclear.iloc[20000:].doc_url.to_list() 

In [56]:
parse_text(links)

### Get Daily Kos Articles

In [9]:
dailykos = newzy[newzy.doc_source == 'Daily Kos'].reset_index()

In [11]:
dailykos.head()

,doc_id,doc_source,doc_title,doc_content,doc_date,doc_url
0,96,Daily Kos,Obama has done nothing to address income inequ...,Income and wealth inequality is a festering wo...,11/03/2013,http://feeds.dailykos.com/~r/dailykos/index/~3...
1,97,Daily Kos,"Anti-racism is not 'anti-white,' no matter wha...",The virulent meme spread across our nation wit...,11/03/2013,http://feeds.dailykos.com/~r/dailykos/index/~3...
2,98,Daily Kos,Abbreviated Pundit Round-up: Angry does no...,Chrystia Freeland sees a fight that's not not ...,11/03/2013,http://feeds.dailykos.com/~r/dailykos/index/~3...
3,99,Daily Kos,Sunday Talk: Haters gonna hate,"No doubt, the bestest thing about America is o...",11/03/2013,http://feeds.dailykos.com/~r/dailykos/index/~3...
4,100,Daily Kos,Gore on fire: Says humans treating atmosphere ...,About time Al Gore has been everywhere lately....,11/03/2013,http://feeds.dailykos.com/~r/dailykos/index/~3...


In [25]:
links = dailykos.doc_url.to_list()

In [26]:
len(links)

12310

In [27]:
parse_text(links)

In [28]:
len(articles)

12310

In [29]:
dailykos['articles'] = articles

In [30]:
dailykos.head()

,doc_id,doc_source,doc_title,doc_content,doc_date,doc_url,articles
0,96,Daily Kos,Obama has done nothing to address income inequ...,Income and wealth inequality is a festering wo...,11/03/2013,http://feeds.dailykos.com/~r/dailykos/index/~3...,But how strong is the correlation between fall...
1,97,Daily Kos,"Anti-racism is not 'anti-white,' no matter wha...",The virulent meme spread across our nation wit...,11/03/2013,http://feeds.dailykos.com/~r/dailykos/index/~3...,The virulent meme spread across our nation wit...
2,98,Daily Kos,Abbreviated Pundit Round-up: Angry does no...,Chrystia Freeland sees a fight that's not not ...,11/03/2013,http://feeds.dailykos.com/~r/dailykos/index/~3...,"Why are the plutocrats, with their great wealt..."
3,99,Daily Kos,Sunday Talk: Haters gonna hate,"No doubt, the bestest thing about America is o...",11/03/2013,http://feeds.dailykos.com/~r/dailykos/index/~3...,"Unfortunately for Barack Obama, a growing numb..."
4,100,Daily Kos,Gore on fire: Says humans treating atmosphere ...,About time Al Gore has been everywhere lately....,11/03/2013,http://feeds.dailykos.com/~r/dailykos/index/~3...,Al Gore has been everywhere lately. Just last ...


In [31]:
dailykos.to_csv('dailykos.csv')

In [62]:
articles = pd.Series(articles)
#articles = articles.to_list()

In [63]:
articles.to_csv('articles.csv')

In [60]:
realclear['articles'] = articles

In [65]:
realclear.to_csv('realclear.csv')